In [22]:
import os
import json
import pandas as pd
import traceback

In [23]:
%pip install langchain langchain-community

from langchain.chat_models import ChatOpenAI

Note: you may need to restart the kernel to use updated packages.


In [24]:
KEY="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"

In [25]:
# # llm=ChatOpenAI(openai_api_key=KEY,model_name="gemini_flash", temperature=0.5)

# llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-4o", temperature=0.5)
KEY="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"
llm= ChatOpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=KEY,
  temperature=0.5,
)


In [26]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D5B8830BB0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D5B8830670>, temperature=0.5, model_kwargs={}, openai_api_key='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', openai_api_base='https://openrouter.ai/api/v1', openai_proxy='')

In [27]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain , SequentialChain 
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [28]:
RESPONSE_JSON = {
	"1": {
		"mcq": "multiple choice question",
		"options": {
			"a": "choice here",
			"b": "choice here",
			"c": "choice here",
			"d": "choice here",
		},
		"correct": "correct answer"
	},
 
 "2": {
		"mcq": "multiple choice question",
		"options": {
			"a": "choice here",
			"b": "choice here",
			"c": "choice here",
			"d": "choice here",
		},
		"correct": "correct answer"
	},
 "3": {
		"mcq": "multiple choice question",
		"options": {
			"a": "choice here",
			"b": "choice here",
			"c": "choice here",
			"d": "choice here",
		},
		"correct": "correct answer"
	},
 "": {
		"mcq": "multiple choice question",
		"options": {
			"a": "choice here",
			"b": "choice here",
			"c": "choice here",
			"d": "choice here",
		},
		"correct": "correct answer"
	}
 
}


In [29]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of fnumber} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [30]:
from langchain.prompts import PromptTemplate

# Fix the template string: replace "fnumber}" with "{number}"
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

quiz_generation_prompt = PromptTemplate(
	input_variables=["text", "number", "subject", "tone", "response_json"],
	template=TEMPLATE
)

In [31]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt,output_key="quiz" ,verbose=True)

In [32]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. \
if the quiz is not at per with the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

"""

In [33]:
quiz_evaluation_prompt = PromptTemplate(input_variables=("subject","quiz"), template=TEMPLATE)

In [34]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review", verbose=True)

In [35]:
genarate_evaluation_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [36]:
file_path = r"C:\Users\EL-Hussein Store\Desktop\MCQ GEN\experment\data.txt"

In [37]:
with open(file_path, "r") as file:
    Text = file.read()
    

In [38]:
Text

'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]\n\nThe earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive systems cont

In [39]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [40]:
number=5
subject="machine learning"
tone="simple" 


In [41]:
with get_openai_callback() as cb:
    response = genarate_evaluation_chain(
        {
            "text": Text,
            "number": number,
            "subject": subject,
            "tone": tone,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons us

AuthenticationError: Error code: 401 - {'error': {'message': 'User not found.', 'code': 401}}

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain , SequentialChain 
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain , SequentialChain 
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain , SequentialChain 
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain , SequentialChain 
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2622
Prompt Tokens:1886
Completion Tokens:736
Total Cost:0.004300999999999999


In [ ]:
response

NameError: name 'response' is not defined

In [ ]:
quiz=response.get("quiz")

In [ ]:
quiz=json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
   mcq = value["mcq"]
   options = " | ".join(
       [
           f"{option}: {option_value}"
           for option, option_value in value["options"].items()
       ]
   )
   
   correct = value["correct"]
   quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [42]:
quiz.to_csv("mcq.csv", index=False)

NameError: name 'quiz' is not defined

In [ ]:
from datetime import datetime
datetime.now()

datetime.datetime(2025, 8, 13, 22, 52, 58, 506824)